In [14]:
# common library
import pandas as pd
import numpy as np
import time
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('.')))
# preprocessor
from finrl.preprocessing.preprocessors import *
# config
from finrl.config.config import *
# model
from finrl.model.models import *
import os

import pdb
pdb.set_trace()

--Call--
> /mnt/work/project/FinRL-Library/.venv/lib/python3.7/site-packages/IPython/core/displayhook.py(252)__call__()
    250         sys.stdout.flush()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.
    254 

self = <ipykernel.displayhook.ZMQShellDisplayHook object at 0x7fdb4029f278>
result = None
> /mnt/work/project/FinRL-Library/.venv/lib/python3.7/site-packages/IPython/core/displayhook.py(258)__call__()
    256         activated by setting the variable sys.displayhook to it.
    257         """
--> 258         self.check_for_underscore()
    259         if result is not None and not self.quiet():
    260             self.start_displayhook()

> /mnt/work/project/FinRL-Library/.venv/lib/python3.7/site-packages/IPython/core/displayhook.py(259)__call__()
    257         """
    258         self.check_for_underscore()
--> 259         if result is not None and not self.quiet():
    260             self.start_displayhook(

#提取数据

In [2]:
import yfinance as yf
from finrl.marketdata.yahoodownloader import YahooDownloader

SAVE_PATH = "./20210616-12h19.csv"
if os.path.exists(SAVE_PATH):
    df = pd.read_csv(SAVE_PATH)
else:
    df = YahooDownloader(
        config.START_DATE,  #'2000-01-01',
        config.END_DATE,  # 2021-01-01，预计将改日期改为'2021-06-20'（今日日期）
        ticker_list=config.DOW_30_TICKER,
    ).fetch_data()  # DOW_30_TICKER)道琼斯30只股票
    df.to_csv(SAVE_PATH)

简单看一下数据

In [3]:
# read and preprocess data
print(df)

        Unnamed: 0        date        open        high         low  \
0                0  1999-12-31    0.901228    0.918527    0.888393   
1                1  1999-12-31   48.287384   49.016800   48.287384   
2                2  1999-12-31   41.250000   41.625000   41.062500   
3                3  1999-12-31   23.000000   23.718750   22.843750   
4                4  1999-12-31   53.343750   53.593750   52.968750   
...            ...         ...         ...         ...         ...   
156481      156481  2020-12-31  218.399994  219.820007  216.199997   
156482      156482  2020-12-31   58.060001   58.799999   58.020000   
156483      156483  2020-12-31   39.330002   40.000000   39.029999   
156484      156484  2020-12-31  144.199997  144.270004  142.850006   
156485      156485  2020-12-31   41.470001   41.680000   40.970001   

             close     volume   tic  day  
0         0.789364  163811200  AAPL    4  
1        36.021698    1164972   AXP    4  
2        26.747131    1111200 

In [4]:
print(df.head())
print(df.size)

   Unnamed: 0        date       open       high        low      close  \
0           0  1999-12-31   0.901228   0.918527   0.888393   0.789364   
1           1  1999-12-31  48.287384  49.016800  48.287384  36.021698   
2           2  1999-12-31  41.250000  41.625000  41.062500  26.747131   
3           3  1999-12-31  23.000000  23.718750  22.843750  13.182354   
4           4  1999-12-31  53.343750  53.593750  52.968750  39.495262   

      volume   tic  day  
0  163811200  AAPL    4  
1    1164972   AXP    4  
2    1111200    BA    4  
3    1525600   CAT    4  
4   10726400  CSCO    4  
1408374


In [5]:
df.sort_values(["date", "tic"]).head()

,Unnamed: 0,date,open,high,low,close,volume,tic,day
0,0,1999-12-31,0.901228,0.918527,0.888393,0.789364,163811200,AAPL,4
1,1,1999-12-31,48.287384,49.016800,48.287384,36.021698,1164972,AXP,4
2,2,1999-12-31,41.250000,41.625000,41.062500,26.747131,1111200,BA,4
3,3,1999-12-31,23.000000,23.718750,22.843750,13.182354,1525600,CAT,4
4,4,1999-12-31,53.343750,53.593750,52.968750,39.495262,10726400,CSCO,4


数据预处理
"""
    preprocess_data()
        main method to do the feature engineering
"""

In [6]:
import itertools
tech_indicators = ["macd", "rsi_30", "cci_30", "dx_30"]

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=tech_indicators,
    use_turbulence=True,
    user_defined_feature=False,
)
##使用finrl.preprocessing.preprocessors中的FeatureEngineer来对股价数据进行预处理

processed = fe.preprocess_data(df)

list_ticker = processed["tic"].unique().tolist()  # 按照processed的"tic"列去重
list_date = list(
    pd.date_range(processed["date"].min(), processed["date"].max()).astype(str)
)  # 成一个固定频率的时间索引
combination = list(itertools.product(list_date, list_ticker))
"""
1.pandas.date_range(start=None, end=None, periods=None, freq='D', tz=None, normalize=False, name=None, closed=None, **kwargs)
由于import pandas as pd,所以也可以写成pd.date_range（start=None, end=None）
该函数主要用于生成一个固定频率的时间索引，使用时必须指定start、end、periods中的两个参数值，否则报错。
2.df.astype('str') #改变整个df变成str数据类型
3.itertools.product(*iterables[, repeat]) # 对应有序的重复抽样过程
  itertools.product(a,b),将a,b元组中的每个分量依次乘开。
"""

processed_full = pd.DataFrame(combination, columns=["date", "tic"]).merge(
    processed, on=["date", "tic"], how="left"
)
"""1.  pd.DataFrame( 某数据集 ，index  ，columns ),给某数据集加上行名index和列名columns
       此处只有pd.DataFrame( 某数据集 ，columns )，第一列加列名date，第二列加列名tic.
   2.  merge(df1,df2,on='key',how)
   按照["date","tic"]为关键字链接，以左边的dataframe为主导，左侧dataframe取全部数据，右侧dataframe配合左边
"""

processed_full = processed_full[processed_full["date"].isin(processed["date"])]
# isin函数，清洗数据，删选过滤掉processed_full中一些行，processed_full新加一列['date']若和processed_full中的['date']不相符合，则被剔除
processed_full = processed_full.sort_values(["date", "tic"])

processed_full = processed_full.fillna(0)
# 对于processed_full数据集中的缺失值使用 0 来填充.
processed_full.sample(5)  # sample（）是random模块中的一个函数，即随机取五个样本展示

Successfully added technical indicators
Successfully added turbulence index


,date,tic,Unnamed: 0,open,high,low,close,volume,day,macd,rsi_30,cci_30,dx_30,turbulence
62783,2005-09-22,TRV,41754.0,41.709999,42.259998,40.669998,28.519093,3715300.0,3.0,-0.109578,49.198894,-149.926540,24.053839,40.626567
37537,2003-06-04,DIS,24889.0,19.730000,20.598120,19.730000,16.525017,14953877.0,2.0,0.389424,62.425133,259.816480,44.036177,17.295201
125524,2011-06-15,CSCO,84370.0,14.960000,14.990000,14.780000,10.980863,92323100.0,2.0,-0.395375,30.342466,-160.166828,53.698979,16.313076
25506,2002-04-29,DD,16855.0,43.893639,44.149830,42.698093,22.780359,1720686.0,0.0,-0.234411,44.151021,-179.891271,28.156506,49.219670
75345,2006-11-15,MCD,50156.0,41.180000,41.700001,40.990002,26.913950,10242900.0,2.0,0.317360,60.513581,-0.943359,20.429443,17.717450


数据划分

In [9]:
train_start = '2009-01-01'
train_end = '2015-10-01'
val_test_start = '2015-10-01'
val_test_end = '2020-07-20'

# 2015/10/01 is the date that validation starts
# 2016/01/01 is the date that real trading starts
# unique_trade_date needs to start from 2015/10/01 for validation purpose
train_period=(train_start, train_end)
unique_trade_date=(val_test_start, val_test_end)
#unique_trade_date = df[(df.datadate > 20151001)&(df.datadate <= 20200720)].datadate.unique()
print(unique_trade_date)

# rebalance_window is the number of months to retrain the model
# validation_window is the number of months to validation the model and select for trading
rebalance_window = 63
validation_window = 63

('2015-10-01', '2020-07-20')


设计环境模型
"""A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

In [16]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces

# shares normalization factor
# 100 shares per trade
HMAX_NORMALIZE = 100
# initial amount of money we have in our account
INITIAL_ACCOUNT_BALANCE=1000000
# total number of stocks in our portfolio
STOCK_DIM = 30
# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001

# turbulence index: 90-150 reasonable threshold
#TURBULENCE_THRESHOLD = 140
REWARD_SCALING = 1e-4

In [13]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
#import pickle

# shares normalization factor
# 100 shares per trade
HMAX_NORMALIZE = 100
# initial amount of money we have in our account
INITIAL_ACCOUNT_BALANCE=1000000
# total number of stocks in our portfolio
STOCK_DIM = 30
# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001

# turbulence index: 90-150 reasonable threshold
#TURBULENCE_THRESHOLD = 140
REWARD_SCALING = 1e-4

In [ ]:
#def __init__(self, df, day = 0, turbulence_threshold=140, iteration=''):
#super(StockEnv, self).__init__()
# #money = 10 , scope = 1
day = 0
df = df
iteration=i
# action_space normalization and shape is STOCK_DIM
action_space = spaces.Box(low = -1, high = 1,shape = (STOCK_DIM,)) 
# Shape = 181: [Current Balance]+[prices 1-30]+[owned shares 1-30] 
# +[macd 1-30]+ [rsi 1-30] + [cci 1-30] + [adx 1-30]
observation_space = spaces.Box(low=0, high=np.inf, shape = (181,))

# load data from a pandas dataframe
data = df.loc[day,:]
terminal = False     
turbulence_threshold = turbulence_threshold
# initalize state
state = [INITIAL_ACCOUNT_BALANCE] + data.adjcp.values.tolist() + [0]*STOCK_DIM + \
    data.macd.values.tolist() + data.rsi.values.tolist() + data.cci.values.tolist() + \
    data.adx.values.tolist()
# initialize reward
    self.reward = 0
    self.turbulence = 0
    self.cost = 0
    self.trades = 0
    # memorize all the total balance change
    self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
    self.rewards_memory = []
    #self.reset()
    self._seed()
    
    self.iteration=iteration


def _sell_stock(self, index, action):
    # perform sell action based on the sign of the action
    if self.turbulence<self.turbulence_threshold:
        if self.state[index+STOCK_DIM+1] > 0:
            #update balance
            self.state[0] += \
            self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
                (1- TRANSACTION_FEE_PERCENT)
            
            self.state[index+STOCK_DIM+1] -= min(abs(action), self.state[index+STOCK_DIM+1])
            self.cost +=self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
                TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            pass
    else:
        # if turbulence goes over threshold, just clear out all positions 
        if self.state[index+STOCK_DIM+1] > 0:
            #update balance
            self.state[0] += self.state[index+1]*self.state[index+STOCK_DIM+1]* \
                            (1- TRANSACTION_FEE_PERCENT)
            self.state[index+STOCK_DIM+1] =0
            self.cost += self.state[index+1]*self.state[index+STOCK_DIM+1]* \
                            TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            pass

def _buy_stock(self, index, action):
    # perform buy action based on the sign of the action
    if self.turbulence< self.turbulence_threshold:
        available_amount = self.state[0] // self.state[index+1]
        # print('available_amount:{}'.format(available_amount))
        
        #update balance
        self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                            (1+ TRANSACTION_FEE_PERCENT)

        self.state[index+STOCK_DIM+1] += min(available_amount, action)
        
        self.cost+=self.state[index+1]*min(available_amount, action)* \
                            TRANSACTION_FEE_PERCENT
        self.trades+=1
    else:
        # if turbulence goes over threshold, just stop buying
        pass
    
def step(self, actions):
    # print(self.day)
    self.terminal = self.day >= len(self.df.index.unique())-1
    # print(actions)

    if self.terminal:
        plt.plot(self.asset_memory,'r')
        plt.savefig('results/account_value_validation_{}.png'.format(self.iteration))
        plt.close()
        df_total_value = pd.DataFrame(self.asset_memory)
        df_total_value.to_csv('results/account_value_validation_{}.csv'.format(self.iteration))
        end_total_asset = self.state[0]+ \
        sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
        #print("previous_total_asset:{}".format(self.asset_memory[0]))           

        #print("end_total_asset:{}".format(end_total_asset))
        #print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):61]))- self.asset_memory[0] ))
        #print("total_cost: ", self.cost)
        #print("total trades: ", self.trades)

        df_total_value.columns = ['account_value']
        df_total_value['daily_return']=df_total_value.pct_change(1)
        sharpe = (4**0.5)*df_total_value['daily_return'].mean()/ \
                df_total_value['daily_return'].std()
        #print("Sharpe: ",sharpe)
        
        #df_rewards = pd.DataFrame(self.rewards_memory)
        #df_rewards.to_csv('results/account_rewards_trade_{}.csv'.format(self.iteration))
        
        # print('total asset: {}'.format(self.state[0]+ sum(np.array(self.state[1:29])*np.array(self.state[29:]))))
        #with open('obs.pkl', 'wb') as f:  
        #    pickle.dump(self.state, f)
        
        return self.state, self.reward, self.terminal,{}

    else:
        # print(np.array(self.state[1:29]))

        actions = actions * HMAX_NORMALIZE
        #actions = (actions.astype(int))
        if self.turbulence>=self.turbulence_threshold:
            actions=np.array([-HMAX_NORMALIZE]*STOCK_DIM)
        begin_total_asset = self.state[0]+ \
        sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
        #print("begin_total_asset:{}".format(begin_total_asset))
        
        argsort_actions = np.argsort(actions)
        
        sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
        buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

        for index in sell_index:
            # print('take sell action'.format(actions[index]))
            self._sell_stock(index, actions[index])

        for index in buy_index:
            # print('take buy action: {}'.format(actions[index]))
            self._buy_stock(index, actions[index])

        self.day += 1
        self.data = self.df.loc[self.day,:]         
        self.turbulence = self.data['turbulence'].values[0]
        #print(self.turbulence)
        #load next state
        # print("stock_shares:{}".format(self.state[29:]))
        self.state =  [self.state[0]] + \
                self.data.adjcp.values.tolist() + \
                list(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]) + \
                self.data.macd.values.tolist() + \
                self.data.rsi.values.tolist() + \
                self.data.cci.values.tolist() + \
                self.data.adx.values.tolist()
        
        end_total_asset = self.state[0]+ \
        sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
        self.asset_memory.append(end_total_asset)
        #print("end_total_asset:{}".format(end_total_asset))
        
        self.reward = end_total_asset - begin_total_asset            
        # print("step_reward:{}".format(self.reward))
        self.rewards_memory.append(self.reward)
        
        self.reward = self.reward*REWARD_SCALING

    return self.state, self.reward, self.terminal, {}

def reset(self):  
    self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
    self.day = 0
    self.data = self.df.loc[self.day,:]
    self.turbulence = 0
    self.cost = 0
    self.trades = 0
    self.terminal = False 
    #self.iteration=self.iteration
    self.rewards_memory = []
    #initiate state
    self.state = [INITIAL_ACCOUNT_BALANCE] + \
                    self.data.adjcp.values.tolist() + \
                    [0]*STOCK_DIM + \
                    self.data.macd.values.tolist() + \
                    self.data.rsi.values.tolist()  + \
                    self.data.cci.values.tolist()  + \
                    self.data.adx.values.tolist() 
        
    return self.state

def render(self, mode='human',close=False):
    return self.state


def _seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

In [18]:
a= 1
print(f"a={a}")

a=1


In [ ]:
#df
#unique_trade_date= unique_trade_date
#rebalance_window = rebalance_window
#validation_window=validation_window